# Выгрузка библиотек

In [94]:
import pandas as pd
import chardet
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import scipy.stats as sts
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
# %matplotlib
warnings.filterwarnings("ignore")

# Задание 1. (Статистика)

Руководство компании обратило внимание на то, что сотрудники старше 35 лет болеют чаще, чем более молодые сотрудники. Кроме этого, среди мужчин количество пропусков рабочих дней в связи с больничным выше, чем среди женщин. В связи с этой ситуацией, руководство организации планирует ввести дополнительные медицинские осмотры среди групп риска. 
Вам необходимо проверить следующие гипотезы:
Мужчины пропускают в течение года более 2 рабочих дней (work_days) по болезни значимо чаще женщин.
Работники старше 35 лет (age) пропускают в течение года более 2 рабочих дней (work_days) по болезни значимо чаще своих более молодых коллег.
Все необходимые данные содержатся в файле «Статистика».

**Решение необходимо предоставить**:   
В виде jupyter notebook (с аккуратно оформленным кодом, графиками и описанной логикой решения). 
В виде дашборда на Streamlit с простым функционалом: 
должна быть возможность загрузить csv, в формате аналогично файлу «Статистика»
должна быть возможность задать параметры age и work_days
указан результат проверки гипотез
указана логика получения результата (должны быть отрисованы графики распределений, указаны критерии проверки (стат. тесты, статистики, уровень значимости т.п.) 

Код проекта должен быть обернут в docker выложен в gitlab/github репозиторий предоставлена ссылка на него. Будет обращаться внимание на качество кода, код стайл, оформление ноутбука, кода и репозитория. Будет плюсом, если дашборд будет поднят на сервере и на него будет предоставлена ссылка. 


# Данные

In [53]:
with open('Статистика.csv', 'rb') as f:
    enc = chardet.detect(f.read())
df = pd.read_csv('Статистика.csv', encoding = enc['encoding'])

In [54]:
print(enc)

{'encoding': 'windows-1251', 'confidence': 0.99, 'language': 'Russian'}


In [55]:
df.head()

,Количество больничных дней,Возраст,Пол
0,5,39,Ж
1,4,54,М
2,4,26,М
3,5,42,М
4,4,29,М


In [58]:
# Для личного удобства (чтобы не переключать каждый раз язык на клавиатуре)

df.columns = ['sick_leave_days', 'age', 'gender']
df.loc[df['gender'] == "Ж", 'gender'] ='F'
df.loc[df['gender'] == "М", 'gender'] ='M'

In [76]:
print("Количество женщин, взявших больничный более 2-ух раз: ", df[(df['gender'] == 'F') & (df['sick_leave_days'] > 2)].shape[0])
print("Количество мужчин, взявших больничный более 2-ух раз: ", df[(df['gender'] == 'M') & (df['sick_leave_days'] > 2)].shape[0])

Количество женщин, взявших больничный более 2-ух раз:  67
Количество мужчин, взявших больничный более 2-ух раз:  84


In [81]:
print("Количество работников младше 35, взявших больничный более 2-ух раз: ", df[(df['age'] <= 35) & (df['sick_leave_days'] > 2)].shape[0])
print("Количество работников старше 35, взявших больничный более 2-ух раз: ", df[(df['age'] > 35) & (df['sick_leave_days'] > 2)].shape[0])

Количество работников младше 35, взявших больничный более 2-ух раз:  36
Количество работников старше 35, взявших больничный более 2-ух раз:  115


Видим, что выборки не очень большие, но, в целом можно попробовать сделать t-test. Для этого проверим выборки на нормальное распределение   
Если распределения не будут нормальными, воспользуемся тестом Манна-Уитни

# Графики

In [95]:
filtered_data = df[(df['sick_leave_days'] > 2)]

## Мужчины vs Женщины

In [99]:
# График распределения sick_leave_days для мужчин и женщин (первоначальный масштаб) ----
male_data   = filtered_data[filtered_data['gender'] == 'M']
female_data = filtered_data[filtered_data['gender'] == 'F']

fig1 = px.histogram(
                        filtered_data, x='sick_leave_days', 
                        color='gender', 
                        title='Распределение sick_leave_days по полу'
                    )

fig1.update_layout(
                        title='Распределение sick_leave_days по полу',
                        xaxis=dict(title='sick_leave_days'),
                        yaxis=dict(title='Частота')
                    )

fig1.show()


На графике плотности особенно видно, что распределения различаются незначительно

In [102]:
# График плотности
fig1 = ff.create_distplot(
    [male_data['sick_leave_days'], female_data['sick_leave_days']],
    group_labels=['Мужчины', 'Женщины'],
    colors=['blue', 'red'],
    show_hist=False  # Hide histograms, show only density curves
)

fig1.update_layout(
    title='Плотность sick_leave_days по полу',
    xaxis=dict(title='sick_leave_days'),
    yaxis=dict(title='Плотность')
)

fig1.show()

# Взрослые vs Молодые

In [104]:
# График распределения sick_leave_days для старше и моложе 35 лет (первоначальный масштаб)
older_data   = filtered_data[filtered_data['age'] > 35]
younger_data = filtered_data[filtered_data['age'] <= 35]

fig2 = go.Figure()
fig2.add_trace(go.Histogram(
                                x=older_data['sick_leave_days'], 
                                opacity=0.5, 
                                name=f'Старше {35} лет'), 
                            )

fig2.add_trace(go.Histogram(
                                x=younger_data['sick_leave_days'], 
                                opacity=0.5, 
                                name=f'Моложе {35} лет')
                            )
fig2.update_layout(
    title='Распределение sick_leave_days по возрасту',
    xaxis=dict(title='sick_leave_days'),
    yaxis=dict(title='Частота')

)

На графике плотности особенно видно, что распределения различаются незначительно

In [106]:
# График плотности
fig1 = ff.create_distplot(
    [older_data['sick_leave_days'], younger_data['sick_leave_days']],
    group_labels=[f'Старше {35}', f'Моложе {35}'],
    colors=['blue', 'red'],
    show_hist=False  # Hide histograms, show only density curves
)

fig1.update_layout(
    title='Плотность sick_leave_days по возрасту',
    xaxis=dict(title='sick_leave_days'),
    yaxis=dict(title='Плотность')
)

# Проверка гипотез


In [107]:
alpha = 0.05

## Мужчины vs Женщины

In [108]:
# Проверка нормальности распределения (t-тест)
# Нулевая гипотеза - распределение нормальное
_, normal_male_p_value   = stats.normaltest(male_data['sick_leave_days'])
_, normal_female_p_value = stats.normaltest(female_data['sick_leave_days'])

print("Проверка нормальности распределения:")
print(f"Мужчины: p-значение = {round(normal_male_p_value, 2)}")
print(f"Женщины: p-значение = {round(normal_female_p_value, 2)}")

# Проверка, является ли распределение нормальным на уровне значимости
if normal_male_p_value > alpha and normal_female_p_value > alpha:
    # Распределения являются нормальными, поэтому можно использовать t-тест
    statistic, p_value = stats.ttest_ind(
                                            male_data['sick_leave_days'], 
                                            female_data['sick_leave_days'],
                                            alternative='greater'
                                        )
    print(f"t-статистика: {statistic}")
    print(f"p-значение: {round(p_value, 2)}")

    if p_value < alpha:
        print("ВЫВОД: Отвергаем нулевую гипотезу. Существуют статистически значимые различия.")
    else:
        print("ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.")
else:
    print("Распределения не являются нормальными, воспользуемся непараметрическим тестом Манна — Уитни.")

    statistic, p_value = stats.mannwhitneyu(
                                                male_data['sick_leave_days'], 
                                                female_data['sick_leave_days'], 
                                                alternative='greater'
                                            )
    print(' ')
    print(f"Статистика: {statistic}")
    print(f"p-значение: {round(p_value, 2)}")

    if p_value < alpha:
        print("ВЫВОД: Отвергаем нулевую гипотезу. Существуют статистически значимые различия.")
    else:
        print("ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.")

Проверка нормальности распределения:
Мужчины: p-значение = 0.0
Женщины: p-значение = 0.01
Распределения не являются нормальными, воспользуемся непараметрическим тестом Манна — Уитни.
 
Статистика: 2829.0
p-значение: 0.48
ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.


## Взрослые vs Молодые

In [109]:
# Вторая гипотеза: Работники старше 35 лет пропускают в течение года более 2 рабочих дней значимо чаще своих более молодых коллег

# Проверка нормальности распределения (t-тест)
_, normal_older_p_value   = stats.normaltest(older_data['sick_leave_days'])
_, normal_younger_p_value = stats.normaltest(younger_data['sick_leave_days'])

print("Проверка нормальности распределения:")
print(f"Старше 35 лет: p-значение = {round(normal_older_p_value, 2)}")
print(f"Моложе 35 лет: p-значение = {round(normal_younger_p_value, 2)}")

# Проверка, является ли распределение нормальным на уровне значимости
if normal_older_p_value > alpha and normal_younger_p_value > alpha:
# Распределения являются нормальными, поэтому можно использовать t-тест
    statistic, p_value = stats.ttest_ind(   older_data['sick_leave_days'], 
                                            younger_data['sick_leave_days'], 
                                            alternative='greater'
                                        )
    print(f"t-статистика: {statistic}")
    print(f"p-значение: {round(p_value, 2)}")

    if p_value < alpha:
            print("ВЫВОД: Отвергаем нулевую гипотезу. Существуют статистически значимые различия.")
    else:
        print("ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.")
else:
    
    print(f"Распределения не являются нормальными, воспользуемся непараметрическим тестом Манна — Уитни.")

    statistic, p_value = stats.mannwhitneyu(
                                                older_data['sick_leave_days'], 
                                                younger_data['sick_leave_days'], 
                                                alternative='greater'
                                            )
    print(' ')
    print(f"Статистика: {statistic}")
    print(f"p-значение: {round(p_value, 2)}")

    if p_value < alpha:
        print("ВЫВОД:Отвергаем нулевую гипотезу. Существуют статистически значимые различия.")
    else:
        print("ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.")

Проверка нормальности распределения:
Старше 35 лет: p-значение = 0.0
Моложе 35 лет: p-значение = 0.02
Распределения не являются нормальными, воспользуемся непараметрическим тестом Манна — Уитни.
 
Статистика: 1888.0
p-значение: 0.8
ВЫВОД: Не отвергаем нулевую гипотезу. Различия статистически незначимы.
